In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
rng = RandomState(3348)
x_right = np.sin(np.linspace(0,2*np.pi,4))[None] * (rng.randn(20,1) * 0.5)
x_rest = np.sin(np.linspace(0,2*np.pi,4))[None] * (rng.randn(20,1) * 2)


In [ ]:
def rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True)),
         
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True))
    )
    
def res_block(n_c, n_i_c):
     return ResidualBlock(
        nn.Sequential(
        nn.Conv2d(n_c, n_i_c, (3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c, (3,1), stride=1, padding=(1,0),bias=True)),
    )

In [ ]:
inputs_a = np_to_var(x_right[:20,None,:,None], dtype=np.float32).cuda()

inputs_b = np_to_var(x_rest[:20,None,:,None], dtype=np.float32).cuda()
inputs = [inputs_a, inputs_b]

In [ ]:
fig, axes = plt.subplots(4,10, figsize=(12,6), sharex=True, sharey=True)
for i_class in range(2):
    for i_example in range(len(inputs[i_class])):
        i_row = i_example // 5
        i_col = i_example % 5
        i_col += i_class * 5
        axes[i_row][i_col].plot(var_to_np(inputs[i_class][i_example]).squeeze(),
                               color=seaborn.color_palette()[i_class])
fig.suptitle('Input signals')

from matplotlib.lines import Line2D
lines = [Line2D([0], [0], color=seaborn.color_palette()[i_class],) for i_class in range(2)]
labels = ['Right', 'Rest',]
axes[0][-1].legend(lines, labels, bbox_to_anchor=(1,1,0,0))

In [ ]:
plt.figure(figsize=(10,6))
for i_class in range(2):
    plt.plot(var_to_np(inputs[i_class].squeeze()).T, color=seaborn.color_palette()[i_class],);
lines = [Line2D([0], [0], color=seaborn.color_palette()[i_class],) for i_class in range(2)]
plt.legend(lines, ['Right', 'Rest',], bbox_to_anchor=(1,1,0,0))
plt.title('Input signals')

In [ ]:
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
set_random_seeds(2019011641, True)
feature_model = nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),
    rev_block(2,32),
    rev_block(2,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),
    rev_block(4,32),
    rev_block(4,32),
    ViewAs((-1,4,1,1), (-1,4))
)
feature_model.cuda()


from braindecode.torch_ext.util import set_random_seeds
adv_model = nn.Sequential(
    nn.Conv2d(1,16, (3,1), stride=1, padding=(1,0),bias=True),
    res_block(16,32),
    res_block(16,32),
    nn.AvgPool2d((2,1)),
    res_block(16,32),
    res_block(16,32),
    ViewAs((-1,16,2,1), (-1,32)),
    )

from spectral_norm import spectral_norm

adv_model = ProjectionDiscriminator(adv_model,32,2,)

adv_model.cuda()


from reversible.training import hard_init_std_mean
n_dims = inputs_b.shape[2]
n_clusters = 2
# will be initialized properly later
means_per_cluster = [th.autograd.Variable(th.zeros(n_dims).cuda(), requires_grad=True)
                     for _ in range(n_clusters)]
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims).cuda(), requires_grad=True)
                    for _ in range(n_clusters)]




from copy import deepcopy
optimizer = th.optim.Adam(
                          [
    {'params': list(feature_model.parameters()),
    'lr': 1e-3,
    'weight_decay': 0},], betas=(0,0.9))

optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster,
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))


optim_adv = th.optim.Adam([{
    'params': adv_model.parameters(),
    'lr': 4e-3, 'weight_decay': 0.00}],#lr 0.0004
                         betas=(0,0.9))



In [ ]:
from reversible.gaussian import get_gauss_samples
from reversible.revnet import invert 
import pandas as pd
from gradient_penalty import gradient_penalty
import time


df = pd.DataFrame()
g_loss = np_to_var([np.nan],dtype=np.float32)
g_grad = np.nan
d_loss = np_to_var([np.nan],dtype=np.float32)
d_grad = np.nan

In [ ]:
def one_nn_score_train():
    outs_per_class =[]
    for i_class in range(len(means_per_cluster)):
        outs_per_class.append(var_to_np(feature_model(inputs[i_class])))

    preds_per_class = []
    for i_class in range(len(means_per_cluster)):
        preds_this_class = []
        for i_example in range(len(outs_per_class[i_class])):
            this_example = outs_per_class[i_class][i_example]
            min_diffs = []
            for j_class in range(len(means_per_cluster)):
                if j_class != i_class:
                    this_ref_examples = outs_per_class[j_class]
                else:
                    this_ref_examples = np.concatenate((outs_per_class[i_class][:i_example],
                           outs_per_class[i_class][i_example+1:]), axis=0)
                diffs = np.sum(np.square(this_ref_examples - this_example), axis=1)
                min_diffs.append(np.min(diffs))
            preds_this_class.append(np.argmin(min_diffs))
        preds_per_class.append(preds_this_class)
    preds_per_class = np.array(preds_per_class)
    return (np.mean(preds_per_class[0] == 0) + np.mean(preds_per_class[1] == 1)) /  2

In [ ]:
n_epochs = 4001
for i_epoch in range(n_epochs):
    start_time = time.time()
    optim_adv.zero_grad()
    optimizer.zero_grad()
    optim_dist.zero_grad()
    for i_class in range(len(inputs)):
        mean = means_per_cluster[i_class]
        std = th.exp(stds_per_cluster[i_class])
        this_inputs = inputs[i_class]
        y = np_to_var([i_class]).cuda()
        samples = get_gauss_samples(20, mean, std)
        inverted = invert(feature_model, samples)
        score_fake = adv_model(inverted, y)
        if (i_epoch % 10) != 0:
            score_real = adv_model(this_inputs, y)
            gradient_loss = gradient_penalty(adv_model, this_inputs, inverted[:(len(this_inputs))], y)
            d_loss = -score_real.mean() + score_fake.mean() + gradient_loss
            d_loss.backward()
            d_grad = np.mean([th.sum(p.grad **2).item() for p in adv_model.parameters()])
        else:
            g_loss = -th.mean(score_fake)
            g_loss.backward()
            g_grad = np.mean([th.sum(p.grad **2).item() for p in feature_model.parameters()])
    if (i_epoch % 10) != 0:
            optim_adv.step()
    else:
        optimizer.step()
        optim_dist.step()
    sample_wd_row = {}
    for i_class in range(len(inputs)):
        this_inputs = inputs[i_class]
        mean = means_per_cluster[i_class]
        std = th.exp(stds_per_cluster[i_class])
        y = np_to_var([i_class]).cuda()
        samples = get_gauss_samples(20, mean, std)

        inverted = invert(feature_model, samples)
        in_np = var_to_np(this_inputs).squeeze()
        fake_np = var_to_np(inverted).squeeze()
        import ot

        dist = np.sum(np.abs(in_np[:,None] - fake_np[None]), axis=2)
        match_matrix = ot.emd([],[], dist)
        cost = np.sum(dist * match_matrix)
        score_fake = adv_model(inverted, y)
        score_real = adv_model(inputs[i_class], y)
        wd_dist = th.mean(score_real) - th.mean(score_fake)
        sample_wd_row.update({
            'wd_dist_' + str(i_class): wd_dist.item() ,
            'sampled_wd' + str(i_class): cost,
            'wd_diff' + str(i_class): cost - wd_dist.item(),
        })
    end_time = time.time()
    epoch_row = {
    'd_loss': d_loss.item(),
    'g_loss': g_loss.item(),
    'o_real': th.mean(score_real).item(),
    'o_fake': th.mean(score_fake).item(),
    'g_grad': g_grad,
    'd_grad': d_grad,
    'runtime': end_time -start_time,
    '1nn_train': one_nn_score_train()}
    epoch_row.update(sample_wd_row)
    df = df.append(epoch_row, ignore_index=True)
    if i_epoch % (n_epochs // 20) == 0:
        
        display_text("Epoch {:d}".format(i_epoch))
        display(df.iloc[-5:])
        print("stds\n", var_to_np(th.exp(th.stack(stds_per_cluster))))
        for i_class in range(len(stds_per_cluster)):
            std = th.exp(stds_per_cluster[i_class])
            mean = means_per_cluster[i_class]
            i_std_2, i_std_1 = np.argsort(var_to_np(std))[::-1][:2]
            feature_a_values = th.linspace(float(mean[i_std_1].data - 2 * std[i_std_1].data),
                                   float(mean[i_std_1].data + 2 * std[i_std_1].data), 9)
            feature_b_values = th.linspace(float(mean[i_std_2].data - 2 * std[i_std_2].data),
                                   float(mean[i_std_2].data + 2 * std[i_std_2].data), 9)
            image_grid = np.zeros((len(feature_a_values), len(feature_b_values), inputs[i_class].shape[2]))

            for i_f_a_val, f_a_val in enumerate(feature_a_values):
                for i_f_b_val, f_b_val in enumerate(feature_b_values):
                    this_out = mean.clone()
                    this_out.data[i_std_1] = f_a_val
                    this_out.data[i_std_2] = f_b_val
                    inverted = var_to_np(invert(feature_model, this_out.unsqueeze(0))[0]).squeeze()

                    image_grid[i_f_a_val, i_f_b_val] = np.copy(inverted)
            fig, axes = plt.subplots(image_grid.shape[0], image_grid.shape[1],
                 sharex=True, sharey=True,
                figsize=(int(image_grid.shape[0] * 2), int(image_grid.shape[1])))
            plt.subplots_adjust(wspace=0, hspace=0)
            for ax, curve  in zip(axes.flatten(), image_grid.reshape(-1, image_grid.shape[-1])):
                ax.plot(curve)
            display(fig)
            plt.close(fig)
        
        for i_class in range(len(means_per_cluster)):
            outs_np = var_to_np(feature_model(inputs[i_class]))
            fig = plt.figure(figsize=(12,2))
            plt.title("Std of different dimensions")
            plt.plot(np.std(outs_np, axis=0))
            plt.xlabel('Output dimension')
            display(fig)
            plt.close(fig)
        
        for i_class in range(len(inputs)):
            y = np_to_var([i_class]).cuda()
            mean = means_per_cluster[i_class]
            std = th.exp(stds_per_cluster[i_class])
            this_inputs = inputs[i_class]
            samples = get_gauss_samples(2000, mean, std)

            inverted = invert(feature_model, samples)
            in_np = var_to_np(this_inputs).squeeze()
            fake_np = var_to_np(inverted).squeeze()
            import ot

            dist = np.sum(np.abs(in_np[:,None] - fake_np[None]), axis=2)
            match_matrix = ot.emd([],[], dist)
            cost = np.sum(dist * match_matrix)
            print(cost)
            score_fake = adv_model(inverted, y)
            score_real = adv_model(inputs[i_class], y)
            wd_dist = th.mean(score_real) - th.mean(score_fake)
            print(wd_dist)

        for i_class in range(len(inputs)):
            y = np_to_var([i_class]).cuda()
            mean = means_per_cluster[1-i_class]
            std = th.exp(stds_per_cluster[1-i_class])
            this_inputs = inputs[i_class]
            samples = get_gauss_samples(2000, mean, std)

            inverted = invert(feature_model, samples)
            in_np = var_to_np(this_inputs).squeeze()
            fake_np = var_to_np(inverted).squeeze()
            import ot

            dist = np.sum(np.abs(in_np[:,None] - fake_np[None]), axis=2)
            match_matrix = ot.emd([],[], dist)
            cost = np.sum(dist * match_matrix)
            print(cost)
            score_fake = adv_model(inverted, y)
            score_real = adv_model(inputs[i_class], y)
            wd_dist = th.mean(score_real) - th.mean(score_fake)
            print(wd_dist)

In [ ]:
n_classes =   len(stds_per_cluster)
image_grid = np.zeros((len(std), len(feature_a_values), n_classes, inputs[i_class].shape[2]))
for i_class in range(len(stds_per_cluster)):
    std = th.exp(stds_per_cluster[i_class])
    mean = means_per_cluster[i_class]
    for i_std in range(len(std)):
        feature_a_values = th.linspace(float(mean[i_std].data - 2 * std[i_std].data),
                               float(mean[i_std].data + 2 * std[i_std].data), 9)
        for i_f_a_val, f_a_val in enumerate(feature_a_values):
            this_out = mean.clone()
            this_out.data[i_std] = f_a_val
            inverted = var_to_np(invert(feature_model, this_out.unsqueeze(0))[0]).squeeze()
            image_grid[i_std, i_f_a_val, i_class] = np.copy(inverted)
fig, axes = plt.subplots(image_grid.shape[0], image_grid.shape[1],
     sharex=True, sharey=True,
    figsize=(int(image_grid.shape[1] * 1.5), int(image_grid.shape[0]) * 1.25))
plt.subplots_adjust(wspace=0, hspace=0.2)
for i_ax, (ax, curves)  in enumerate(zip(axes.flatten(), 
                                         image_grid.reshape(-1, image_grid.shape[-2], image_grid.shape[-1]))):
    for i_class in range(n_classes):
        ax.plot(np.linspace(0,1000, len(curves[i_class])), 
                curves[i_class], color=seaborn.color_palette()[i_class])
axes[0][-1].legend(['Right', 'Rest'], bbox_to_anchor=[1,1,0,0])
from matplotlib.colors import  LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list(
        'test', seaborn.color_palette()[:2][::-1])
min_ratio, max_ratio = np.sort(var_to_np(th.exp(stds_per_cluster[0]) /
                                         th.exp(stds_per_cluster[1])))[[0,-1]]
for i_std in range(len(stds_per_cluster[0])):
    std1 = th.exp(stds_per_cluster[0][i_std]).item()
    std2 = th.exp(stds_per_cluster[1][i_std]).item()
    ratio = std1/std2
    axes[i_std][0].text(-1000,0,"{:.1f}\n{:.1f}\n{:.1f}".format(
        std1,
        std2,
        ratio), va='center', color=cmap((ratio - min_ratio) / max_ratio))

axes[0][0].text(-1000,2, "Stds\nRatio")
display(fig)
plt.close(fig)


In [ ]:
n_steps = 17
alphas = th.linspace(-0.5,1.5,n_steps).cuda()
mean_dir = means_per_cluster[1] - means_per_cluster[0]

vals = means_per_cluster[0].unsqueeze(0) + (alphas.unsqueeze(1) * mean_dir.unsqueeze(0))

inverted = invert(feature_model, vals).squeeze()


alphas =np.linspace(0,1,len(inverted))
colors = (1 - alphas)[:,None] * np.array(seaborn.color_palette()[0])[None] + (
    alphas[:,None] * np.array(seaborn.color_palette()[1])[None])
plt.figure(figsize=(12,6))
for i_line in range(len(colors)):
    lw = 0.7
    if i_line in [4,17-4]:
        lw=2
    label = ''
    if i_line == 4:
        label = 'Right'
    elif i_line == (17-4):
        label = 'Rest'
    plt.plot(var_to_np(inverted)[i_line], color=colors[i_line], lw=lw, label=label);
plt.title("Morph between two class means")
plt.legend()

In [ ]:
n_steps = 17
alphas = np.linspace(-0.5,1.5,n_steps)
mean_dir = np.mean(var_to_np(inputs[1]).squeeze(), axis=0) - np.mean(var_to_np(inputs[0]).squeeze(), axis=0)

inverted = np.mean(var_to_np(inputs[0]).squeeze(), axis=0)[None] + (alphas[:,None] * mean_dir[None])


alphas =np.linspace(0,1,len(inverted))
colors = (1 - alphas)[:,None] * np.array(seaborn.color_palette()[0])[None] + (
    alphas[:,None] * np.array(seaborn.color_palette()[1])[None])
plt.figure(figsize=(12,6))
for i_line in range(len(colors)):
    lw = 0.7
    if i_line in [4,17-4]:
        lw=2
    label = ''
    if i_line == 4:
        label = 'Right'
    elif i_line == (17-4):
        label = 'Rest'
    plt.plot(inverted[i_line], color=colors[i_line], lw=lw, label=label);
plt.title("Morph between two class means in input space")
plt.legend()

In [ ]:
plt.figure(figsize=(6,2))

for i_class in range(len(means_per_cluster)):
    mean = means_per_cluster[i_class]
    inverted = invert(feature_model, get_gauss_samples(1, mean, mean*0)).squeeze()
    plt.plot(var_to_np(inverted),
            color=seaborn.color_palette()[i_class],
            label=['Right', 'Rest'][i_class] + ' Fake Mode')

    data_mean = np.mean(var_to_np(inputs[i_class]), axis=0).squeeze()
    plt.plot(data_mean,
            color=seaborn.color_palette()[i_class],
             ls='--', label=['Right', 'Rest'][i_class] + ' Real Mean')
plt.legend(bbox_to_anchor=(1,1,0,0))
plt.title("Compare inverted modes and real means")

In [ ]:
for i_class in range(2):
    mean = means_per_cluster[i_class]
    std = th.exp(stds_per_cluster[i_class])
    samples = get_gauss_samples(5000, mean, std)
    inverted = var_to_np(invert(feature_model, samples).squeeze())
    ins = var_to_np(inputs[i_class].squeeze())

    dist_matrix = np.sqrt(np.sum(np.square(ins[:,None] - inverted[None]), axis=2))

    coupling = ot.emd([],[], dist_matrix)
    fig, axes = plt.subplots(4,5, figsize=(16,8), sharex=True, sharey=True)
    for i_in in range(len(ins)):
        mask = coupling[i_in] > 1 / (len(inverted) + 1)
        matched_samples = inverted[mask]
        reference = ins[i_in]
        ax = axes.flatten()[i_in]
        ax.plot(np.array(matched_samples).T, color=seaborn.color_palette()[0], alpha=0.7, lw=0.75);
        ax.plot(reference, color='black')

In [ ]:
# 